# FunctionGemma Fine-tune cho PvZ Bot

Train AI quyết định: `plant(plant_type, row, col)` hoặc `wait()`

**Game State format:**
- `PLANTS:[(type,row,col),...]`
- `ZOMBIES:[(type,row,col),...]`
- `SEEDS:[(type,status),...]` (status: ready/cooldown)

**Output:** PyTorch + OpenVINO IR format

## Workflow:
1. Upload `training_data.json`
2. Train model
3. Export OpenVINO
4. Download

## 1. Cài đặt

In [ ]:
!pip install torch transformers datasets accelerate trl protobuf sentencepiece openvino optimum[openvino] -q

## 2. Upload Training Data

In [ ]:
from google.colab import files
import json

print("Upload training_data.json...")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
with open(filename, 'r') as f:
    raw_data = json.load(f)

print(f"\n✓ Loaded {len(raw_data)} samples")
stats = {}
for s in raw_data:
    stats[s['action']] = stats.get(s['action'], 0) + 1
print(f"  Actions: {stats}")

## 3. Login HuggingFace

In [ ]:
from huggingface_hub import login
login()

## 4. Load Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import get_json_schema

BASE_MODEL = "google/functiongemma-270m-it"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float32,
    device_map="auto",
    attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print(f"✓ Model loaded on {model.device}")

## 5. Define Tools

In [ ]:
def plant(plant_type: str, row: int, col: int) -> str:
    """Plant at grid position. Args: plant_type (pea_shooter/sunflower/wall_nut), row (0-4), col (0-8)"""
    return "Planted"

def wait() -> str:
    """Wait. Use when seed cooldown or not enough sun."""
    return "Waiting"

TOOLS = [get_json_schema(plant), get_json_schema(wait)]
print("✓ Tools:", [t['function']['name'] for t in TOOLS])

## 6. Prepare Dataset

In [ ]:
from datasets import Dataset
import random

SYSTEM_MSG = """PvZ bot. Choose action based on game state.
- PLANTS: planted plants (type,row,col)
- ZOMBIES: zombies (type,row,col)
- SEEDS: seed packets (type,status)
Plant when seed ready. Wait when cooldown."""

def create_conversation(sample):
    action = sample["action"]
    args = sample["arguments"]
    
    if action == "plant":
        tool_call = {"type": "function", "function": {"name": "plant", "arguments": args}}
    else:
        tool_call = {"type": "function", "function": {"name": "wait", "arguments": {}}}
    
    return {
        "messages": [
            {"role": "developer", "content": SYSTEM_MSG},
            {"role": "user", "content": sample["game_state"]},
            {"role": "assistant", "tool_calls": [tool_call]},
        ],
        "tools": TOOLS
    }

random.shuffle(raw_data)
dataset = Dataset.from_list(raw_data)
dataset = dataset.map(create_conversation, remove_columns=dataset.features)
dataset = dataset.train_test_split(test_size=0.2, shuffle=True)
print(f"✓ Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")

## 7. Training

In [ ]:
from trl import SFTTrainer, SFTConfig

num_samples = len(raw_data)
epochs = max(10, 100 // num_samples * 10)

args = SFTConfig(
    output_dir="pvz_gemma",
    max_length=512,
    num_train_epochs=epochs,
    per_device_train_batch_size=4,
    logging_steps=10,
    eval_strategy="epoch",
    learning_rate=5e-5,
    lr_scheduler_type="constant",
    report_to="none",
)

trainer = SFTTrainer(
    model=model, args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
)

print(f"Training {num_samples} samples for {epochs} epochs...")
trainer.train()
print("\n✓ Training complete!")

## 8. Test Model

In [ ]:
def test_bot(game_state):
    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": game_state},
    ]
    inputs = tokenizer.apply_chat_template(messages, tools=TOOLS, add_generation_prompt=True, return_dict=True, return_tensors="pt")
    out = model.generate(**inputs.to(model.device), pad_token_id=tokenizer.eos_token_id, max_new_tokens=64)
    return tokenizer.decode(out[0][len(inputs["input_ids"][0]):], skip_special_tokens=False)

print("="*50)
print("TEST PVZ BOT")
print("="*50)

test_cases = [
    "PLANTS:[]. ZOMBIES:[]. SEEDS:[(pea_shooter,ready)]",
    "PLANTS:[(pea_shooter,2,0)]. ZOMBIES:[(zombie,2,7)]. SEEDS:[(pea_shooter,cooldown)]",
    "PLANTS:[(pea_shooter,2,0),(pea_shooter,2,1)]. ZOMBIES:[(zombie,1,6)]. SEEDS:[(pea_shooter,ready)]",
]

for t in test_cases:
    print(f"\n📥 {t}")
    print(f"📤 {test_bot(t)}")

## 9. Save PyTorch Model

In [ ]:
model.save_pretrained("pvz_gemma_pytorch")
tokenizer.save_pretrained("pvz_gemma_pytorch")
print("✓ PyTorch model saved")

## 10. Export to OpenVINO IR

In [ ]:
from optimum.intel import OVModelForCausalLM

print("Converting to OpenVINO IR format...")
ov_model = OVModelForCausalLM.from_pretrained(
    "pvz_gemma_pytorch",
    export=True,
    compile=False
)
ov_model.save_pretrained("pvz_gemma_openvino")
tokenizer.save_pretrained("pvz_gemma_openvino")
print("✓ OpenVINO model saved to pvz_gemma_openvino/")

## 11. Test OpenVINO Model

In [ ]:
from optimum.intel import OVModelForCausalLM

ov_model = OVModelForCausalLM.from_pretrained("pvz_gemma_openvino")
ov_tokenizer = AutoTokenizer.from_pretrained("pvz_gemma_openvino")

def test_ov(game_state):
    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": game_state},
    ]
    inputs = ov_tokenizer.apply_chat_template(messages, tools=TOOLS, add_generation_prompt=True, return_dict=True, return_tensors="pt")
    out = ov_model.generate(**inputs, pad_token_id=ov_tokenizer.eos_token_id, max_new_tokens=64)
    return ov_tokenizer.decode(out[0][len(inputs["input_ids"][0]):], skip_special_tokens=False)

print("\n" + "="*50)
print("TEST OPENVINO MODEL")
print("="*50)

for t in test_cases:
    print(f"\n📥 {t}")
    print(f"📤 {test_ov(t)}")

## 12. Download Models

In [ ]:
# Zip both models
!zip -r pvz_gemma_pytorch.zip pvz_gemma_pytorch/
!zip -r pvz_gemma_openvino.zip pvz_gemma_openvino/

print("\n✓ Models ready for download:")
print("  - pvz_gemma_pytorch.zip (HuggingFace format)")
print("  - pvz_gemma_openvino.zip (OpenVINO IR format)")

In [ ]:
from google.colab import files

# Download OpenVINO model (recommended for inference)
files.download('pvz_gemma_openvino.zip')

In [ ]:
# Optional: Download PyTorch model
files.download('pvz_gemma_pytorch.zip')